In [17]:
# Import Libraries and Set API Key
import openai
import pandas as pd
import json
from dotenv import load_dotenv
import os
from openai import OpenAI


# Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")



In [18]:
# Load Processed
df = pd.read_csv("../data/processed/cleaned_tweets.csv")
df_validation = pd.read_csv("../data/processed/cleaned_tweets_validation.csv")
df.head()
df_validation.head()

,sentiment,text,cleaned_text
0,Positive,im getting on borderlands and i will murder yo...,im getting borderlands murder
1,Positive,I am coming to the borders and I will kill you...,coming borders kill
2,Positive,im getting on borderlands and i will kill you ...,im getting borderlands kill
3,Positive,im coming on borderlands and i will murder you...,im coming borderlands murder
4,Positive,im getting on borderlands 2 and i will murder ...,im getting borderlands murder


In [19]:
# Prepare Data for OpenAI Fine-Tuning (JSONL format)
def create_jsonl(df, filename):
    with open(filename, 'w') as file:
        for _, row in df.iterrows():
            json_record = {
                "messages": [
                    {"role": "system", "content": "You classify tweet sentiment as positive, negative, or neutral."},
                    {"role": "user", "content": row['cleaned_text']},
                    {"role": "assistant", "content": row['sentiment']}
                ]
            }
            file.write(json.dumps(json_record) + "\n")

create_jsonl(df, "../data/processed/train_data.jsonl")
create_jsonl(df_validation, "../data/processed/test_data.jsonl")


In [ ]:
client = OpenAI()
# Upload the file
file = client.files.create(
    file=open("../data/processed/train_data.jsonl", "rb"),
    purpose="fine-tune"
)

# Initiate fine-tuning
fine_tune_job = client.fine_tuning.jobs.create(
    training_file=file.id,
    model="gpt-3.5-turbo"
)


file-FwPsPbkuGxuFUwP3WciZoF
